# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [5]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [6]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [7]:
# TODO: Create a .env file with the following variables
# OPENAI_API_KEY="YOUR_KEY"
# CHROMA_OPENAI_API_KEY="YOUR_KEY"
# TAVILY_API_KEY="YOUR_KEY"

In [8]:
# TODO: Load environment variables
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
tavily_api_key = os.getenv('TAVILY_API_KEY') 
chroma_openai_api_key = os.getenv('CHROMA_OPENAI_API_KEY')

for key in [openai_api_key, tavily_api_key, chroma_openai_api_key]:
    assert key is not None

### VectorDB Instance

In [9]:
# TODO: Instantiate your ChromaDB Client
# Choose any path you want
chroma_client = chromadb.PersistentClient(path="chromadb")

### Collection

In [10]:
# TODO: Pick one embedding function
# If picking something different than openai, 
# make sure you use the same when loading it
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai_api_key,
    model_name="text-embedding-3-small",
    api_base="https://openai.vocareum.com/v1"
)

In [11]:
# TODO: Create a collection
# Choose any name you want
try: 
   collection = chroma_client.create_collection(
      name="udaplay",
      embedding_function=embedding_fn
   )
except:
   collection = chroma_client.get_collection(
      name="udaplay",
      embedding_function=embedding_fn
   )

### Add documents

In [11]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]
    
    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

### Semantic Search

In [12]:
query = collection.query(
    query_texts=["A game about space exploration and survival"],
    n_results=3
)

for i, doc in enumerate(query['documents'][0]):
    print(f"Result {i+1}:\n{doc}\n")

Result 1:
[Xbox One] Minecraft (2014) - A sandbox game that allows players to build and explore infinite worlds, fostering creativity and adventure.

Result 2:
[Xbox Series X|S] Halo Infinite (2021) - The latest installment in the Halo franchise, featuring Master Chief's return in a new open-world setting.

Result 3:
[PlayStation 2] Grand Theft Auto: San Andreas (2004) - An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson.

